[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/simple_rag.ipynb)

# 简单RAG（检索增强生成）系统

## 概述

此代码实现了一个基本的检索增强生成（RAG）系统，用于处理和查询PDF文档。该系统将文档内容编码到向量存储中，然后可以查询以检索相关信息。

## 核心组件

1. PDF处理和文本提取
2. 文本分块以便于处理
3. 使用[FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/)和OpenAI嵌入创建向量存储
4. 设置检索器以查询处理后的文档
5. RAG系统评估

## 方法详情

### 文档预处理

1. 使用PyPDFLoader加载PDF。
2. 使用RecursiveCharacterTextSplitter将文本分割成指定大小和重叠的块。

### 文本清理

应用自定义函数`replace_t_with_space`来清理文本块。这可能解决PDF中的特定格式问题。

### 向量存储创建

1. 使用OpenAI嵌入创建文本块的向量表示。
2. 从这些嵌入创建FAISS向量存储，以实现高效的相似性搜索。

### 检索器设置

1. 配置检索器以获取给定查询的前2个最相关的块。

### 编码函数

`encode_pdf`函数封装了加载、分块、清理和将PDF编码到向量存储的整个过程。

## 主要特性

1. 模块化设计：编码过程封装在单个函数中，便于重用。
2. 可配置分块：允许调整块大小和重叠。
3. 高效检索：使用FAISS进行快速相似性搜索。
4. 评估：包含评估RAG系统性能的函数。

## 使用示例

代码包含一个测试查询："气候变化的主要原因是什么？"。这演示了如何使用检索器从处理后的文档中获取相关上下文。

## 评估

系统包含一个`evaluate_rag`函数来评估检索器的性能，尽管提供的代码中没有详细说明使用的具体指标。

## 此方法的优势

1. 可扩展性：通过分块处理可以处理大型文档。
2. 灵活性：易于调整块大小和检索结果数量等参数。
3. 效率：利用FAISS在高维空间中进行快速相似性搜索。
4. 与先进NLP集成：使用OpenAI嵌入进行最先进的文本表示。

## 结论

这个简单的RAG系统为构建更复杂的信息检索和问答系统提供了坚实的基础。通过将文档内容编码到可搜索的向量存储中，它能够高效地检索相关信息以响应查询。这种方法对于需要快速访问大型文档或文档集合中特定信息的应用程序特别有用。

# 软件包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [1]:
# Install required packages
%pip install pypdf==5.6.0
%pip install PyMuPDF==1.26.1
%pip install python-dotenv==1.1.0
%pip install langchain-community==0.3.25
%pip install langchain_openai==0.3.23
%pip install rank_bm25==0.2.2
%pip install faiss-cpu==1.11.0
%pip install deepeval==3.1.0

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https:/

In [2]:
# # Clone the repository to access helper functions and evaluation modules
# !git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
# import sys
# sys.path.append('RAG_TECHNIQUES')

# # If you need to run with the latest data
# # !cp -r RAG_TECHNIQUES/data .

In [3]:
import os
import sys
from dotenv import load_dotenv
# from google.colab import userdata

sys.path.append(os.path.dirname(os.path.abspath('.')))

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable (comment out if not using OpenAI)
# if not userdata.get('SF_API_KEY'):
#     os.environ["SF_API_KEY"] = input("Please enter your OpenAI API key: ")
# else:
#     os.environ["SF_API_KEY"] = userdata.get('SF_API_KEY')

# Original path append replaced for Colab compatibility



from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from helper_functions import (EmbeddingProvider,
                              retrieve_context_per_question,
                              replace_t_with_space,
                              get_langchain_embedding_provider,
                              show_context)

# from evaluation.evalute_rag import evaluate_rag
 
from langchain.vectorstores import FAISS


### Read Docs

In [4]:
# # 下载所需的数据文件
# import os
# os.makedirs('data', exist_ok=True)

# # 下载本笔记本中使用的PDF文档
# !wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
# !wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [5]:
# path = "../data/Understanding_Climate_Change.pdf"
path = "../data/my_resume.pdf"

### 编码文档

In [6]:
def encode_pdf(path, chunk_size=1000, chunk_overlap=200):
    """
    使用OpenAI嵌入将PDF书籍编码到向量存储中。

    Args:
        path: PDF文件的路径。
        chunk_size: 每个文本块的期望大小。
        chunk_overlap: 连续块之间的重叠量。

    Returns:
        包含编码书籍内容的FAISS向量存储。
    """

    # 加载PDF文档
    loader = PyPDFLoader(path)
    documents = loader.load()

    # 将文档分割成块
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # 创建嵌入（已测试OpenAI和Amazon Bedrock）
    embeddings = get_langchain_embedding_provider(EmbeddingProvider.OPENAI, model_id="Qwen/Qwen3-Embedding-4B"
    , base_url=os.environ["SF_BASE_URL"], api_key=os.environ["SF_API_KEY"])
    #embeddings = get_langchain_embedding_provider(EmbeddingProvider.AMAZON_BEDROCK)

    # 创建向量存储
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

In [30]:
chunks_vector_store = encode_pdf(path, chunk_size=500, chunk_overlap=100)

### 创建检索器

In [31]:
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 2})

In [32]:
# 获取向量存储中的总文档数
total_chunks = chunks_vector_store.index.ntotal
print(f"总分块数量: {total_chunks}")

总分块数量: 12


In [ ]:
def get_chunk_by_index(vector_store, index):
    """根据索引获取特定分块"""
    docstore_id = vector_store.index_to_docstore_id[index]
    document = vector_store.docstore.search(docstore_id)
    return document

# 使用示例
chunk = get_chunk_by_index(chunks_vector_store, 8)  # 获取第一个分块
print(f"分块内容: {chunk.page_content}")
print(f"分块元数据: {chunk.metadata}")

### 测试检索器

In [ ]:
test_query = "利用  asyncio 对整个告警处理流程进行异步化改造 ， 显著提升了系统的并发处理能力和响应速度"
context = retrieve_context_per_question(test_query, chunks_query_retriever)
show_context(context)

### 评估结果

In [ ]:
# # 注意 - 目前仅适用于OPENAI
# evaluate_rag(chunks_query_retriever)